In [1]:
import pandas as pd
import sklearn
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt

In [2]:
dataset= pd.read_csv('C:\Users\Admin\Downloads\survey_deleted00.csv')

In [3]:
dataset.shape

(713, 17)

In [4]:
dataset= dataset.rename(columns={'Timestamp':'timestamp',
                            'When something good happens to me, I have people who I like to share the good news with.	':'Q1',
                            'I finish whatever I begin.	':'Q2',
                           'When I do an	activity, I	enjoy it so much that I	lose track of time.	':'Q3',
                           'I get completely absorbed	in what I	am doing.	':'Q4',
                           'I am optimistic about my future	':'Q5',
                           'I keep at	my collegework until I	am	done with it.	':'Q6',
                           'When I have a problem, I have someone	who will be there for me.	':'Q7',
                           'In uncertain times, I expect the	best.	':'Q8',
                           'There are people in my life	who really care about me.	':'Q9',
                           'I think good things are going to	happen to me.	':'Q10',
                           'I have friends that I really care about.	':'Q11',
                           'Once I make a plan to	get something done, I stick to it.':'Q12',
                           'I believe	that	things will work out, no matter how difficult they seem.':'Q13',
                           'I work hard.	':'Q14',
                           'I have a lot of fun':'Q15',
                           'From the below categories which one do you identify with the most':'label'})

In [5]:
dataset[:1]

,timestamp,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,label
0,2017/01/30 11:01:20 AM GMT+5:30,Very Often,Sometimes,Almost never,Almost never,Almost never,Very Often,Almost always,Almost never,Almost always,Almost never,Very Often,Sometimes,Almost never,Almost always,Often,Mentally distressed


In [6]:
dataset=dataset.drop('timestamp', axis=1)

In [7]:
dataset.label[dataset.label=="Mentally distressed"] = 0
dataset.label[dataset.label=="unaffected with life (neutral)"] = 1
dataset.label[dataset.label=="Optimistic and happy with life"] = 2

In [8]:
dataset[dataset=='Almost always'] = 2
dataset[dataset=='Very Often'] = 1
dataset[dataset=='Often'] = 0
dataset[dataset=='Sometimes'] = -1
dataset[dataset=='Almost never'] = -2


In [9]:
dataset = dataset.astype(int)

In [10]:

Y = np.array(dataset.label)
X = np.array(dataset.drop('label',axis=1))

In [11]:
from sklearn.model_selection import train_test_split
train_x, validation_x, train_y, validation_y = train_test_split(X, Y, test_size=0.2, random_state= 0)

In [12]:
from sklearn import linear_model
logistic = linear_model.LogisticRegression()
logistic.fit(train_x, train_y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [13]:
output_logistic = logistic.predict(validation_x)
np.mean(output_logistic == validation_y)  

0.80419580419580416

In [14]:
from sklearn.naive_bayes import BernoulliNB

clf = BernoulliNB()
clf.fit(train_x, train_y)
BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)
output_ber = clf.predict(validation_x)
np.mean(output_ber == validation_y) 

0.72027972027972031

In [15]:
from sklearn.svm import SVC
svm = SVC(probability=True)
svm.fit(train_x, train_y)
output_svm = svm.predict(validation_x)
np.mean(output_svm == validation_y)

0.87412587412587417

In [16]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators = 200, oob_score = True, n_jobs = -1,random_state =50,max_features = "auto",min_samples_leaf = 2)
rf.fit(train_x,train_y) 
output_rf = rf.predict(validation_x)
np.mean(output_rf == validation_y)

0.87412587412587417

In [17]:
from sklearn.ensemble import AdaBoostClassifier
ab = AdaBoostClassifier()
ab.fit(train_x,train_y)
output_ab = ab.predict(validation_x)
np.mean(output_ab == validation_y)


0.78321678321678323

In [23]:
output_ensemble = []   #ensemble
for i in range(0,len(validation_y)):
    count_0 = 0
    count_1 = 0
    count_2 = 0
    if(output_logistic[i] == '0'): 
        count_0 +=1
    elif(output_logistic[i] == '1'):
        count_1 +=1
    elif(output_logistic[i] == '2'):
        count_2 +=1
    if(output_svm[i] == '0'): 
        count_0 +=1
    elif(output_svm[i] == '1'):
        count_1 +=1
    elif(output_svm[i] == '2'):
        count_2 +=1 
   
    
    if(count_0 >= count_1 and count_0 >= count_2): 
        output_ensemble.append(0)
    elif(count_1 >= count_0 and count_1 >= count_2): 
        output_ensemble.append(1)
    elif(count_2 >= count_0 and count_2 >= count_1): 
        output_ensemble.append(2)
output = np.asarray(output_ensemble)
np.mean(output == validation_y)
    
    
    

0.37062937062937062

In [19]:
class Config:
    nn_input_dim = 2
    nn_output_dim = 2
    epsilon = 0.01
    reg_lambda = 0.01

In [20]:
def generate_data():
    np.random.seed(0)
    X, y = datasets.make_moons(200,noise = 0.20)
    return X ,y

In [21]:
from sklearn.metrics import precision_recall_fscore_support

In [22]:
precision_recall_fscore_support(validation_y,output_rf)

(array([ 0.96      ,  0.83636364,  0.81578947]),
 array([ 0.90566038,  0.90196078,  0.79487179]),
 array([ 0.93203883,  0.86792453,  0.80519481]),
 array([53, 51, 39], dtype=int64))